In [61]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from PIL import Image
import matplotlib.pyplot as plt
import random
import pathlib
import pandas as pd
import torch
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision import transforms
from utils import get_accuracy, plot_curve, keep_store_dict, store_dict_to_disk

ImportError: cannot import name 'get_accuracy' from 'utils' (C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\utils\__init__.py)

In [11]:
filename='mpii_new.csv'
df = pd.read_csv(filename)
df

,ID,NAME,r ankle_X,r ankle_Y,r knee_X,r knee_Y,r hip_X,r hip_Y,l hip_X,l hip_Y,...,r shoulder_Y,l shoulder_X,l shoulder_Y,l elbow_X,l elbow_Y,l wrist_X,l wrist_Y,Scale,Activity,Category
0,5,060111501.jpg,980,322,896,318,865,248,943,226,...,147,923,123,995,163,961,223,3.806403,curling,sports
1,7,002058449.jpg,918,456,659,518,713,413,979,288,...,270,877,247,1112,384,1012,489,8.280870,curling,sports
2,14,029122914.jpg,-1,-1,301,473,302,346,362,345,...,220,371,213,396,309,393,290,2.204083,"truck driving, loading and unloading truck, ty...",occupation
3,15,061185289.jpg,-1,-1,-1,-1,-1,-1,-1,-1,...,288,552,355,-1,-1,-1,-1,4.143112,"truck driving, loading and unloading truck, ty...",occupation
4,16,013949386.jpg,-1,-1,-1,-1,110,385,208,355,...,303,212,153,319,123,376,39,4.431105,"truck driving, loading and unloading truck, ty...",occupation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11426,17367,050240222.jpg,607,689,599,542,605,416,691,410,...,232,663,228,623,289,555,333,3.480000,pushing car,transportation
11427,17368,033474347.jpg,876,461,815,380,779,296,742,303,...,214,663,223,674,286,631,260,2.003677,pushing car,transportation
11428,17369,082650067.jpg,1038,656,950,504,903,328,831,337,...,204,729,220,704,332,675,269,3.235820,pushing car,transportation
11429,17370,072772110.jpg,562,617,558,539,553,452,511,458,...,377,490,378,470,411,461,414,1.494952,pushing car,transportation


In [57]:
class ImageDataset(Dataset):
    def __init__(self, heat_maps_dir, img_dir, transform, target_transform=None):
        self.heat_maps_dir = heat_maps_dir
        self.img_dir = img_dir
        self.transform = transform = transforms.Compose([
            transforms.Resize(transform),
            ])
        self.target_transform = None

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, os.listdir(self.img_dir)[idx])
        image = read_image(img_path)

        map_path = os.path.join(self.heat_maps_dir, os.listdir(self.heat_maps_dir)[idx])
        mapa = np.load(map_path)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            mapa = self.target_transform(mapa)
        return image, mapa

In [59]:
dataset_test = ImageDataset("heat_maps_test_plain", "images_test", (368, 368))
dataset_training = ImageDataset("heat_maps_training_plain", "images_training", (368, 368))

In [33]:
""" data_augmentation = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),  # 0.1 radians is approximately 10 degrees
    transforms.RandomResizedCrop(368, scale=(0.9, 1.0), ratio=(1.0, 1.0))  # Assuming 10% zoom translates to a scale of (0.9, 1.0)
]) """

In [3]:
def gaussian_initializer(mean=0.0, stddev=0.01):
    def _initializer(tensor):
        return nn.init.normal_(tensor, mean=mean, std=stddev)
    return _initializer

def constant_initializer(value=0.0):
    def _initializer(tensor):
        return nn.init.constant_(tensor, value)
    return _initializer

In [10]:
class ModelAVE(nn.Module):
    def __init__(self):
        super(ModelAVE, self).__init__()
        self.avg_pool = nn.AvgPool2d(kernel_size=9, stride=8)

    def forward(self, x):
        center_map = x[:, 3:, :, :]
        return self.avg_pool(center_map)

In [39]:
class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        self.conv1_stage1 = nn.Conv2d(3, 128, kernel_size=9, padding=4)
        self.conv2_stage1 = nn.Conv2d(128, 128, kernel_size=9, padding=4)
        self.conv3_stage1 = nn.Conv2d(128, 128, kernel_size=9, padding=4)
        self.conv4_stage1 = nn.Conv2d(128, 32, kernel_size=5, padding=2)
        self.conv5_stage1 = nn.Conv2d(32, 512, kernel_size=9, padding=4)
        self.conv6_stage1 = nn.Conv2d(512, 512, kernel_size=1)
        self.conv7_stage1 = nn.Conv2d(512, 15, kernel_size=1)
        self.pool_center_lower = None
        
    def _initialize_weights(self):
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    gaussian_initializer()(m.weight)
                    if m.bias is not None:
                        constant_initializer()(m.bias)

    def forward(self, x):
        image = x[:, :3, :, :]
        x1 = F.relu(self.conv1_stage1(image))
        x1 = F.max_pool2d(x1, kernel_size=3, stride=2)
        x1 = F.relu(self.conv2_stage1(x1))
        x1 = F.max_pool2d(x1, kernel_size=3, stride=2)
        x1 = F.relu(self.conv3_stage1(x1))
        x1 = F.max_pool2d(x1, kernel_size=3, stride=2)
        x1 = F.relu(self.conv4_stage1(x1))
        x1 = F.relu(self.conv5_stage1(x1))
        x1 = F.relu(self.conv6_stage1(x1))
        x1 = self.conv7_stage1(x1)
        return x1

In [40]:
class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        self.conv1_stage2 = nn.Conv2d(3, 128, kernel_size=9, padding=4)
        self.conv2_stage2 = nn.Conv2d(128, 128, kernel_size=9, padding=4)
        self.conv3_stage2 = nn.Conv2d(128, 128, kernel_size=9, padding=4)
        self.conv4_stage2 = nn.Conv2d(128, 32, kernel_size=5, padding=2)

    def _initialize_weights(self):
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    gaussian_initializer()(m.weight)
                    if m.bias is not None:
                        constant_initializer()(m.bias)

    def forward(self, x):
        image = x[:, :3, :, :]
        x2 = F.relu(self.conv1_stage2(image))
        x2 = F.max_pool2d(x2, kernel_size=3, stride=2)
        x2 = F.relu(self.conv2_stage2(x2))
        x2 = F.max_pool2d(x2, kernel_size=3, stride=2)
        x2 = F.relu(self.conv3_stage2(x2))
        x3 = F.max_pool2d(x2, kernel_size=3, stride=2)
        x2 = F.relu(self.conv4_stage2(x3))
        return x2, x3

In [41]:
class Concatenated(nn.Module):
    def __init__(self, model1, model2, modelAVE):
        super(Concatenated, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.modelAVE = modelAVE
    
    def forward(self, x):
        image = x[:, :3, :, :]
        center_map = x[:, 3:, :, :]
        x1 = self.model1(image)
        x2 = self.model2(image)
        x_ave = self.modelAVE(center_map)
        concatenated_output = torch.cat([x1, x2, x_ave], dim=1)
        return concatenated_output

In [42]:
class ModelM(nn.Module):
    def __init__(self):
        super(ModelM, self).__init__()
        self.Mconv1 = nn.Conv2d(79, 128, kernel_size=11, padding=5)
        self.Mconv2 = nn.Conv2d(128, 128, kernel_size=11, padding=5)
        self.Mconv3 = nn.Conv2d(128, 128, kernel_size=11, padding=5)
        self.Mconv4 = nn.Conv2d(128, 128, kernel_size=1)
        self.Mconv5 = nn.Conv2d(128, 15, kernel_size=1)

    def _initialize_weights(self):
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    gaussian_initializer()(m.weight)
                    if m.bias is not None:
                        constant_initializer()(m.bias)

    def forward(self, x):
        image = x[:, :3, :, :]
        x2 = F.relu(self.Mconv1_stage2(image))
        x2 = F.relu(self.Mconv2_stage2(x2))
        x2 = F.relu(self.Mconv3_stage2(x2))
        x2 = F.relu(self.Mconv4_stage2(x2))
        x2 = F.relu(self.Mconv5_stage2(x2))
        return x2

In [43]:
class Model3(nn.Module):
    def __init__(self):
        super(Model3, self).__init__()
        self.conv1_stage3 = nn.Conv2d(32, 32, kernel_size=5, padding=2)
        

    def _initialize_weights(self):
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    gaussian_initializer()(m.weight)
                    if m.bias is not None:
                        constant_initializer()(m.bias)

    def forward(self, x):
        image = x[:, :3, :, :]
        x3 = F.relu(self.conv1_stage3(image))
        return x3

In [52]:
model1 = Model1()
model2 = Model2()
modelAVE = ModelAVE()
concatenated_stage2 = Concatenated(model1, model2, modelAVE)
model2_M = ModelM()
model3 = Model3()
concatenated_stage3 = Concatenated(model3, model2_M, modelAVE)
model3_M = ModelM()

In [53]:
class Model(nn.Module):
    def __init__(self, model1, model2, model3, modelAVE, concatenated_stage2, concatenated_stage3, model2_M, model3_M):
        super(Model, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
        self.modelAVE = modelAVE
        self.concatenated2 = concatenated_stage2
        self.concatenated3 = concatenated_stage3
        self.model2_M = model2_M
        self.model3_M = model3_M 
        

    def _initialize_weights(self):
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    gaussian_initializer()(m.weight)
                    if m.bias is not None:
                        constant_initializer()(m.bias)
    
    def forward(self, x):
        output1 = self.model1(x)
        output2, input5 = self.model2(x)
        outputAVE = self.modelAVE(x)
        concatenated_output_stage2 = self.concatenated2(output1, output2, outputAVE)
        output3 = self.model2_M(concatenated_output_stage2)     
        output4 = self.model3(input5)
        concatenated_output_stage3 = self.concatenated3(output3, outputAVE, output4)
        output5 = self.model3_M
        concatenated_output = torch.cat([output1, output2, output3,concatenated_output_stage2, outputAVE, concatenated_output_stage2, concatenated_output_stage3, output4, output5], dim=1)
        return concatenated_output
        

In [54]:
model = Model(model1, model2, model3, modelAVE, concatenated_stage2, concatenated_stage3, model2_M, model3_M)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

Model(
  (model1): Model1(
    (conv1_stage1): Conv2d(3, 128, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
    (conv2_stage1): Conv2d(128, 128, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
    (conv3_stage1): Conv2d(128, 128, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
    (conv4_stage1): Conv2d(128, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (conv5_stage1): Conv2d(32, 512, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
    (conv6_stage1): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1))
    (conv7_stage1): Conv2d(512, 15, kernel_size=(1, 1), stride=(1, 1))
  )
  (model2): Model2(
    (conv1_stage2): Conv2d(3, 128, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
    (conv2_stage2): Conv2d(128, 128, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
    (conv3_stage2): Conv2d(128, 128, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
    (conv4_stage2): Conv2d(128, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  )
  (model3): Model

In [55]:
loss_function = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
def test(model, test_loader):
    # Put the model in evaluation mode. 
    # Tells the model not to compute gradients. Increases inference speed.
    model.eval()
    test_acc = 0.0
    with torch.no_grad():
        for batch_num, (x, y) in enumerate(test_loader, 0):
            # Put the data to the appropriate device.
            x = x.to(device)
            y = y.to(device)
            # Do inference. Forwad pass with the model.
            y_hat = model(x)
            test_acc += get_accuracy(y_hat, y)
    return test_acc / batch_num

In [ ]:
def train(model, num_epochs, train_loader, store_dict, test_loader=None):
    for epoch in range(num_epochs):
        train_running_loss = 0.0
        train_acc = 0.0

        model = model.train()
        
        for batch_num, (x, y) in enumerate(train_loader):
            
            x = x.to(device)
            y = y.to(device)

            y_hat = model(x)
            loss = loss_function(y_hat, y)
            optimizer.zero_grad()
            # Do backpropagation algorithm and calculate all relevant gradients.
            loss.backward()
            # Update model parameters (weights and biases) w.r.t. computed gradients.
            optimizer.step()

            train_running_loss += loss.detach().item()
            train_acc += get_accuracy(y_hat=y_hat, y=y)

        
        epoch_loss = train_running_loss / batch_num
        epoch_acc = train_acc / batch_num
        
        store_dict = keep_store_dict(curve=epoch_loss, label='train_loss', store_dict=store_dict)
        store_dict = keep_store_dict(curve=epoch_acc, label='train_acc', store_dict=store_dict)
        print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
            %(epoch, epoch_loss, epoch_acc))

        if test_loader is not None:
            test_acc = test(model=model, test_loader=test_loader)
            store_dict = keep_store_dict(curve=test_acc, label='test_acc', store_dict=store_dict)
        
    return store_dict

In [ ]:
store_dict = train(model=model, num_epochs=5, train_loader=dataset_training, test_loader=dataset_test, store_dict=None)
store_dict_to_disk(store_dict=store_dict)